In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Read tables
Read caselist from file "study.caselist" and the combination of results and parameters from file "collect.csv".

In [2]:
caselist=pd.read_csv('C:/Users/olawid/PycharmProjects/psm/doc/demo/study_box_fullfact/study.caselist', 
                     index_col=0, delim_whitespace=True)
collect=pd.read_csv('C:/Users/olawid/PycharmProjects/psm/doc/demo/study_box_fullfact/collect.csv', 
                    sep=';', index_col=0)

In [3]:
caselist[:5]

,density,height,width,length
CASENAME,,,,
1,1049.95,1.6998,4.999,13.998
2,1049.95,1.6998,4.999,12.000
3,1049.95,1.6998,4.999,10.002
4,1049.95,1.6998,4.000,13.998
5,1049.95,1.6998,4.000,12.000


In [4]:
collect[:5]

,base_area,mass,volume,density,height,width,length
CASENAME,,,,,,,
1,69.976002,124886.521349,118.945208,1049.95,1.6998,4.999,13.998
2,59.988000,107060.884140,101.967602,1049.95,1.6998,4.999,12.000
3,49.999998,89235.246931,84.989997,1049.95,1.6998,4.999,10.002
4,55.992000,99929.202920,95.175202,1049.95,1.6998,4.000,13.998
5,48.000000,85665.840480,81.590400,1049.95,1.6998,4.000,12.000


A table of results can be created from the collect data, by excluding from it the parameter columns found in the caselist.

In [5]:
results=collect[[c for c in collect.columns if c not in caselist.columns]]

In [6]:
results[:5]

,base_area,mass,volume
CASENAME,,,
1,69.976002,124886.521349,118.945208
2,59.988000,107060.884140,101.967602
3,49.999998,89235.246931,84.989997
4,55.992000,99929.202920,95.175202
5,48.000000,85665.840480,81.590400


# Analysis of variance
We can compute some basic statistics for both parameters and results.

## Parameters

In [7]:
caselist.describe().T[['mean','std','max','min']]

,mean,std,max,min
density,1000.0,50.658546,1049.9500,950.0500
height,1.5,0.202634,1.6998,1.3002
width,4.0,0.827251,4.9990,3.0010
length,12.0,1.654501,13.9980,10.0020


## Results

In [8]:
results.describe().T[['mean','std','max','min']]

,mean,std,max,min
base_area,48.0,12.006862,69.976002,30.016002
mass,72000.0,20954.538494,124886.521349,37077.416851
volume,72.0,20.608963,118.945208,39.026806


# Analysis
## Normalization of data

In [9]:
spar=(caselist-caselist.mean())/caselist.std()
sres=(results-results.mean())/results.std()

## Ordinary Linear Regression
We select one of the output variables by index.

In [10]:
iout = 0
y=sres[sres.columns[iout]]

Regression...

In [11]:
mod = sm.OLS(y, sm.add_constant(spar))
reg = mod.fit()
reg.summary()

C:\Users\olawid\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              base_area   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     605.7
Date:                Fri, 05 Jul 2019   Prob (F-statistic):           6.11e-29
Time:                        22:26:01   Log-Likelihood:                 28.111
No. Observations:                  36   AIC:                            -46.22
Df Residuals:                      31   BIC:                            -38.30
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.724e-16      0.020  -3.38e-14      1.000      -0.041       0.041
density             0      0.020          0      1.000      -0.041       0.041
height      1.076e-16      0.020   5.33e-15      1.000      -0.041       0.041
width          0.8268      0.020     40.956      0.000       0.786       0.868
length         0.5512      0.020     27.304      0.000       0.510       0.592
==============================================================================
Omnibus:                        1.015   Durbin-Watson:                   1.500
Prob(Omnibus):                  0.602   Jarque-Bera (JB):                0.844
Skew:                          -0.000   Prob(JB):                        0.656
Kurtosis:                       2.250   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Re-scale coefficents and standard error to original quantities:

In [12]:
coeff=reg.params*results.std()[iout]/caselist.std()
se=reg.bse*results.std()[iout]/caselist.std()

Compute main effects, in terms of original model parameters:

In [13]:
main_effect=coeff*caselist.std()

Assemble analysis output table, with parameters sorted by significance (low P-values):

In [14]:
out=pd.DataFrame({'params':reg.params, 'bse':reg.bse, 'pvalues':reg.pvalues, 'coeff':coeff, 'se':se, 'main_effect':main_effect})
out=out.drop('const')
out=out.sort_values(by='pvalues')
out

,params,bse,pvalues,coeff,se,main_effect
width,8.267778e-01,0.020187,1.471107e-28,1.200000e+01,0.293001,9.927007e+00
length,5.511852e-01,0.020187,3.020349e-23,4.000000e+00,0.146500,6.618005e+00
height,1.076260e-16,0.020187,1.000000e+00,6.377256e-15,1.196171,1.292250e-15
density,0.000000e+00,0.020187,1.000000e+00,0.000000e+00,0.004785,0.000000e+00
